In [1]:
import pandas as pd
df = pd.read_csv('./data/final_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,1,6448,1,0.001139,0.000891,2,11,16,11,7,0,0
1,1,1376,1,0.001139,0.000643,4,11,16,7,9,0,1
2,1,1883,1,0.001139,0.000957,2,11,16,14,48,0,1
3,1,6716,1,0.001139,0.004782,2,11,16,78,51,0,0
4,1,4710,1,0.001139,0.000575,3,11,16,5,16,0,0


In [9]:
df.shape

(11366, 12)

In [2]:
# standarlize
x = df['Page_Rank_Src']
df['Page_Rank_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Page_Rank_Dst']
df['Page_Rank_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Shortest_Path']
df['Shortest_Path'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Dst']
df['Followees_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Src']
df['Followees_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Dst']
df['Followers_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Src']
df['Followers_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followees']
df['Int_Followees'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followers']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,1,6448,1,0.039538,0.029343,0.500000,0.039568,0.101266,0.039568,0.044304,0.0,0.000000
1,1,1376,1,0.039538,0.019138,0.833333,0.039568,0.101266,0.025180,0.056962,0.0,0.037037
2,1,1883,1,0.039538,0.032066,0.500000,0.039568,0.101266,0.050360,0.303797,0.0,0.037037
3,1,6716,1,0.039538,0.189417,0.500000,0.039568,0.101266,0.280576,0.322785,0.0,0.000000
4,1,4710,1,0.039538,0.016343,0.666667,0.039568,0.101266,0.017986,0.101266,0.0,0.000000


In [3]:
from sklearn.model_selection import train_test_split
df_x = df.drop(columns=['Source', 'Destination', 'Class'])
df_y = df[['Class']]
x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = 0.3, random_state=42)

In [4]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

estimators = [50, 100, 150]
depths = [9, 12, 15, 24]
split = np.random.randint(100, 150, 2)
leaves = np.random.randint(20, 30, 2)
param_grid = {'n_estimators' : estimators, 'max_depth' : depths, 'min_samples_split' : split, 'min_samples_leaf' : leaves}

gs_rf = GridSearchCV(cv = 5, estimator = RandomForestClassifier(), param_grid = param_grid)
gs_rf.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 12, 15, 24],
                         'min_samples_leaf': array([29, 23]),
                         'min_samples_split': array([109, 107]),
                         'n_estimators': [50, 100, 150]})

In [7]:
import pickle

pickle.dump(gs_rf.best_estimator_, open('./data/rf_model.p', 'wb')) 
print(gs_rf.best_params_)

{'max_depth': 15, 'min_samples_leaf': 23, 'min_samples_split': 109, 'n_estimators': 50}


In [8]:
from sklearn.metrics import classification_report
print(classification_report(gs_rf.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       0.87      0.84      0.86      1725
           1       0.84      0.87      0.86      1685

    accuracy                           0.86      3410
   macro avg       0.86      0.86      0.86      3410
weighted avg       0.86      0.86      0.86      3410

